In [12]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

In [24]:
import calendar
import dateutil.parser as parser
from langchain.tools import tool
import time
@tool("weekday")
def weekday(date_str: str) -> str:
    """Convert date to weekday name"""
    d = parser.parse(date_str)
    time.sleep(2)
    return calendar.day_name[d.weekday()]

tools = [weekday]  # 将自定义的 tool 添加到 tools 数组中

In [14]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
        model="ep-20250209190728-xzmmk",
        base_url="https://ark.cn-beijing.volces.com/api/v3/",
        api_key="da677bb3-2da7-4217-963b-90b324875011",
        temperature=0.1,
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# ---------------- 提示词模板定义 ----------------
# 定义系统消息
system_message = SystemMessagePromptTemplate.from_template(
    "你是一个搜索日期的智能机器人xsun。必须使用提供的工具来回答问题。"
)

# 定义工具描述
tools_description = """
你有以下工具可用：
{tools}
"""

# 定义用户消息模板
human_template = """
Answer the following questions as best you can. Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action，如果其中有日期，请确保只输入日期，格式为:YYYY-MM-DD，不要有任何其它字符
Observation: the result of the action，如果其中有日期，请确保输出的日期格式为:YYYY-MM-DD，不要有任何其它字符
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Let's think step by step. Take a deep breath.

Question: {input}
Thought:{agent_scratchpad}
"""

# 组合提示词模板
prompt = ChatPromptTemplate.from_messages([
    system_message,
    SystemMessagePromptTemplate.from_template(tools_description),
    HumanMessagePromptTemplate.from_template(human_template)
])

In [ ]:
from typing_extensions import TypedDict
from langgraph.managed import IsLastStep
from langgraph.graph.message import add_messages
from typing import Annotated
from langchain.schema import BaseMessage


   

In [22]:
from langgraph.checkpoint.memory import MemorySaver


In [23]:
graph = create_react_agent(model, tools, prompt=prompt,checkpointer=MemorySaver())
graph_png = graph.get_graph().draw_mermaid_png()
with open("langgraph_bbzl.png", "wb") as f:
    f.write(graph_png)